In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
! pip install lightfm

     |████████████████████████████████| 307kB 4.8MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707629 sha256=997e651f28371aa41adcb14bbb0ce27bec23351face2e7353b9feaf2d47ee7f6
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [4]:
import pandas as pd
import numpy as np
import os

In [5]:
path='/content/gdrive/My Drive/Project Candy/'
candy_filename = 'candy.csv'
df_candy = pd.read_csv(
    os.path.join(path, candy_filename))

In [6]:
df_candy

,item,user,review
0,Reese's Peanut Butter Cups Miniatures,darlene90,5
1,Reese's Peanut Butter Cups Miniatures,taylordarlene,4
2,Reese's Peanut Butter Cups Miniatures,aliciadennis,5
3,Reese's Peanut Butter Cups Miniatures,ocook,5
4,Reese's Peanut Butter Cups Miniatures,chad38,5
...,...,...,...
17229,Hershey's Nuggets Chocolate Assortment,acostanoah,5
17230,Hershey's Nuggets Chocolate Assortment,jeffrey01,5
17231,Hershey's Nuggets Chocolate Assortment,powellmegan,5
17232,Hershey's Nuggets Chocolate Assortment,benjaminjohnson,5


In [9]:
user_list=np.sort(df_candy['user'].unique())
item_list = df_candy['item'].unique()

In [10]:
user_list.size

2531

In [11]:
item_list.size

142

In [12]:
def id_mappings(user_list, item_list):
    """
    
    Create id mappings to convert user_id, item_id, and feature_id
    
    """
    user_to_index_mapping = {}
    index_to_user_mapping = {}
    for user_index, user_id in enumerate(user_list):
        user_to_index_mapping[user_id] = user_index
        index_to_user_mapping[user_index] = user_id
        
    item_to_index_mapping = {}
    index_to_item_mapping = {}
    for item_index, item_id in enumerate(item_list):
        item_to_index_mapping[item_id] = item_index
        index_to_item_mapping[item_index] = item_id
        
        
        
    return user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping


In [13]:
user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping = id_mappings(user_list, item_list)


In [14]:
from scipy.sparse import coo_matrix
def get_interaction_matrix(df, df_column_as_row, df_column_as_col, df_column_as_value, row_indexing_map, 
                          col_indexing_map):
    
    row = df[df_column_as_row].apply(lambda x: row_indexing_map[x]).values
    col = df[df_column_as_col].apply(lambda x: col_indexing_map[x]).values
    value = df[df_column_as_value].values
    
    return coo_matrix((value, (row, col)), shape = (len(row_indexing_map), len(col_indexing_map)))

In [15]:
user_interaction = get_interaction_matrix(df_candy, "user", 
                                                    "item", "review", user_to_index_mapping, item_to_index_mapping)

In [16]:
from lightfm.cross_validation import random_train_test_split
train,test=random_train_test_split(user_interaction, test_percentage=0.2, random_state=None)

In [17]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))


Precision: train 0.23, test 0.04.
AUC: train 0.83, test 0.68.


In [18]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.30, test 0.05.
AUC: train 0.90, test 0.75.


In [19]:
model = LightFM(learning_rate=0.05, loss='warp')
model.fit(user_interaction,epochs=10)

In [29]:
def sample_recommendation(model, user_interaction, user):
    n_users, n_items = user_interaction.shape
    
    userindex = user_to_index_mapping.get(user, None)
        
    if userindex == None:
      return None


    scores = model.predict(userindex, np.arange(n_items))
    top_items = item_list[np.argsort(-scores)]

    known_positives=df_candy[df_candy['user']==user]['item']

    print("User %s" % user)
    print("   --Rated Candies:")

    for x in known_positives[:5]:
      print("        %s" % x)

    print("   --Recommended Candies:")

    for x in top_items[:5]:
      print("        %s" % x)

In [30]:
sample_recommendation(model, user_interaction, 'ocook')

User ocook
   --Rated Candies:
        Reese's Peanut Butter Cups Miniatures
        Hershey's Cookies 'n' Creme Candy Bar
        Swedish Fish Red Candy
        Twix
        Hershey's Milk Chocolate Bar
   --Recommended Candies:
        Hershey's Kisses Milk Chocolate
        Twix
        Hershey's Cookies 'n' Creme Candy Bar
        Snickers Chocolate Bar
        Starburst Original Fruit Chews


In [31]:
sample_recommendation(model, user_interaction, 'darlene90')

User darlene90
   --Rated Candies:
        Reese's Peanut Butter Cups Miniatures
        Hershey's Natural Unsweetened Cocoa
        Haribo Happy Cola Gummi Candy
        Hershey's Cookies 'n' Creme Candy Bar
        Trolli Sour Brite Crawlers
   --Recommended Candies:
        Hershey's Cookies 'n' Creme Candy Bar
        Reese's Peanut Butter Cups Miniatures
        Ferrero Rocher Chocolate
        Hershey's Kisses Milk Chocolate
        Twix
